# Preprocessing datasets for RGI7 (RGI6 too for testing)

In [1]:
import os 
import glob # finds all the pathnames matching a specified pattern according to the rules used by the Unix shell
import numpy as np
import pandas as pd
import geopandas as gpd

import rasterio
from rasterio.merge import merge
from rasterio.plot import show
from rasterio.enums import Resampling

# Import libraries
import rioxarray as riox
from rioxarray.merge import merge_arrays


## Preprocess Millan et al. 2022 dataset

In [2]:
# list_files = glob.glob("/home/rooda/Dropbox/Patagonia/GIS South/Glaciers/Millan_2022/THICKNESS*.tif")
list_files = glob.glob("/Users/milliespencer/Desktop/data_for_preprocessing/M22/Millan_2022_data/RGI-17_thickness/THICKNESS_RGI-17.2_2022August24.tif")
# there's two tif files (17.2 and 17.3) for my region... can i merge both? also do I disregard the error files? 

In [3]:
glacier_list = []

# Read rasters file
for glacier in list_files:
    glacier_i = riox.open_rasterio(glacier)
    # glacier_i = glacier_i.rio.reproject("EPSG:32718")
    glacier_i = glacier_i.rio.reproject("EPSG:32719") # my region is slightly further east 
    glacier_list.append(glacier_i)

# Merge/Mosaic multiple rasters using merge_arrays method of rioxarray
merged_raster = merge_arrays(dataarrays = glacier_list, res = (50, 50), crs="EPSG:32719", method='max')
merged_raster = merged_raster.round(0).astype("int16")

# Save Raster to disk
# merged_raster.rio.to_raster("/home/rooda/Dropbox/Patagonia/GIS South/Glaciers/Thickness_2022.tif")
merged_raster.rio.to_raster("/Users/milliespencer/Desktop/data_for_preprocessing/M22/Millan_2022_data/Thickness_2022.tif")

## Preprocess Farinotti et al. 2019 dataset

In [4]:
# Rodrigo suggests RGI6 not 7 
# RGI6 = gpd.read_file("/home/rooda/Dropbox/Patagonia/GIS South/Glaciers/RGI6.shp")
RGI6 = gpd.read_file("/Users/milliespencer/Desktop/data_for_preprocessing/RGI6/17_rgi60_SouthernAndes/17_rgi60_SouthernAndes.shp")

# RGI6 = RGI6[RGI6.CenLat < -40.5]  # only glaciers in southern Chile-Argentina
RGI6 = RGI6[(RGI6.CenLat >= -40) & (RGI6.CenLat <= -36.13)]
# made it 36.13 because I feel like there's already a lot of studies on Glacier Universidad, plus it's not a volcano 
# alternatively could make the region the border of Maule or Nuble instead of a latitude line... open to thoughts 

In [5]:
print(RGI6)

                RGIId         GLIMSId   BgnDate   EndDate   CenLon   CenLat  \
12431  RGI60-17.12432  G287959E39962S  20009999  20030531 -72.0414 -39.9622   
12432  RGI60-17.12433  G287982E39961S  20009999  20030531 -72.0181 -39.9609   
12433  RGI60-17.12434  G287983E39959S  20009999  20030531 -72.0170 -39.9588   
12434  RGI60-17.12435  G287956E39958S  20009999  20030531 -72.0443 -39.9583   
12435  RGI60-17.12436  G287955E39956S  20009999  20030531 -72.0450 -39.9555   
...               ...             ...       ...       ...      ...      ...   
13245  RGI60-17.13246  G288972E36148S  20009999  20030531 -71.0285 -36.1481   
13246  RGI60-17.13247  G288985E36146S  20009999  20030531 -71.0155 -36.1455   
13247  RGI60-17.13248  G288978E36145S  20009999  20030531 -71.0219 -36.1453   
13248  RGI60-17.13249  G288982E36137S  20009999  20030531 -71.0180 -36.1371   
13249  RGI60-17.13250  G288977E36135S  20009999  20030531 -71.0228 -36.1351   

      O1Region O2Region   Area  Zmin  ...  Aspect  

In [7]:
# Rodrigo suggests RGI6 not 7 
# RGI6 = gpd.read_file("/home/rooda/Dropbox/Patagonia/GIS South/Glaciers/RGI6.shp")
RGI6 = gpd.read_file("/Users/milliespencer/Desktop/data_for_preprocessing/RGI6/17_rgi60_SouthernAndes/17_rgi60_SouthernAndes.shp")

# RGI6 = RGI6[RGI6.CenLat < -40.5]  # only glaciers in southern Chile-Argentina
RGI6 = RGI6[(RGI6.CenLat >= -40) & (RGI6.CenLat <= -36.13)]
# made it 36.13 because I feel like there's already a lot of studies on Glacier Universidad, plus it's not a volcano 
# alternatively could make the region the border of Maule or Nuble instead of a latitude line... open to thoughts 
print(RGI6) # huh, still 818 rows - why?


                RGIId         GLIMSId   BgnDate   EndDate   CenLon   CenLat  \
12431  RGI60-17.12432  G287959E39962S  20009999  20030531 -72.0414 -39.9622   
12432  RGI60-17.12433  G287982E39961S  20009999  20030531 -72.0181 -39.9609   
12433  RGI60-17.12434  G287983E39959S  20009999  20030531 -72.0170 -39.9588   
12434  RGI60-17.12435  G287956E39958S  20009999  20030531 -72.0443 -39.9583   
12435  RGI60-17.12436  G287955E39956S  20009999  20030531 -72.0450 -39.9555   
...               ...             ...       ...       ...      ...      ...   
13245  RGI60-17.13246  G288972E36148S  20009999  20030531 -71.0285 -36.1481   
13246  RGI60-17.13247  G288985E36146S  20009999  20030531 -71.0155 -36.1455   
13247  RGI60-17.13248  G288978E36145S  20009999  20030531 -71.0219 -36.1453   
13248  RGI60-17.13249  G288982E36137S  20009999  20030531 -71.0180 -36.1371   
13249  RGI60-17.13250  G288977E36135S  20009999  20030531 -71.0228 -36.1351   

      O1Region O2Region   Area  Zmin  ...  Aspect  

In [9]:
# Rodrigo suggests RGI6 not 7 
# RGI6 = gpd.read_file("/home/rooda/Dropbox/Patagonia/GIS South/Glaciers/RGI6.shp")
RGI6 = gpd.read_file("/Users/milliespencer/Desktop/data_for_preprocessing/RGI6/17_rgi60_SouthernAndes/17_rgi60_SouthernAndes.shp")

# RGI6 = RGI6[RGI6.CenLat < -40.5]  # only glaciers in southern Chile-Argentina
RGI6 = RGI6[(RGI6.CenLat >= -40) & (RGI6.CenLat <= -36.13)]
# made it 36.13 because I feel like there's already a lot of studies on Glacier Universidad, plus it's not a volcano 
# alternatively could make the region the border of Maule or Nuble instead of a latitude line... open to thoughts 

# split list ## what's this part? not sure what this is doing 
def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))

# File and folder paths
# list_files = glob.glob("/home/rooda/Dropbox/Patagonia/GIS South/Glaciers/Farinotti_2019/RGI*.tif")
list_files = glob.glob("/Users/milliespencer/Desktop/data_for_preprocessing/F19/RGI60-17/*.tif") # looks for all tif files that start with RGI in this folder 
list_files_df = pd.DataFrame(list_files,  columns =['dir']) # Creates a DataFrame from list_files
list_files_df["RGIId"] = list_files_df.dir.str[64:78] # extracts characters from index 64 to 77 from each file path string. The extracted substring is assigned to a new column named RGIId.

list_files_df = list_files_df[list_files_df['RGIId'].isin(RGI6.RGIId)]
list_files = list_files_df.dir.tolist() # 12000 glaciers

list_files_split = list(split(list_files, 5)) # maximum number of glaciers: 4000 app

In [34]:
print(list_files)
#hmm, that feels problematic - why is it empty? 

[]


In [10]:
for i in range(0,5): # divide in several grops (5 in this case)
    sublist = list_files_split[i]
    glacier_list = []

    for glacier in sublist:
        glacier_i = riox.open_rasterio(glacier)
        glacier_i = glacier_i.rio.reproject("EPSG:32719", resampling = Resampling.bilinear)
        glacier_list.append(glacier_i)
        
    merged_raster = merge_arrays(dataarrays = glacier_list, res = (50, 50), crs="EPSG:32719", method='max')
    merged_raster.rio.to_raster("/Users/milliespencer/Desktop/data_for_preprocessing/RGI6/group_" + str(i) + ".tif")
    print(i)

IndexError: list index out of range

In [ ]:
list_files = glob.glob("/home/rooda/Dropbox/Patagonia/GIS South/Glaciers/group_*.tif") # merge 5 groups

glacier_list = []

# Read rasters file
for glacier in list_files:
    glacier_i = riox.open_rasterio(glacier)
    glacier_i = glacier_i.rio.reproject("EPSG:32718", resampling = Resampling.bilinear)
    glacier_list.append(glacier_i)

# Merge/Mosaic multiple rasters using merge_arrays method of rioxarray
merged_raster = merge_arrays(dataarrays = glacier_list, res = (100, 100), crs="EPSG:32718", method='max')

# Save Raster to disk
merged_raster.rio.to_raster("/home/rooda/Dropbox/Patagonia/GIS South/Glaciers/Thickness_2019.tif")

In [ ]:
for file in list_files: # remove temporary files
    os.remove(file)

## Preprocess Huggonet et al. 2021 dataset

In [ ]:
## dhdt
list_files = glob.glob("/home/rooda/Dropbox/Patagonia/GIS South/Glaciers/dhdt/*.tif")

glacier_list = []

# Read rasters file
for glacier in list_files:
    glacier_i = riox.open_rasterio(glacier)
    glacier_i = glacier_i.rio.reproject("EPSG:32718", resampling = Resampling.bilinear)
    glacier_list.append(glacier_i)

# Merge/Mosaic multiple rasters using merge_arrays method of rioxarray
merged_raster = merge_arrays(dataarrays = glacier_list, res = (100, 100), crs="EPSG:32718", method='max')

# Save Raster to disk
merged_raster.rio.to_raster("/home/rooda/Dropbox/Patagonia/GIS South/Glaciers/dhdt_2021.tif")

In [ ]:
# dhdt_error
list_files = glob.glob("/home/rooda/Dropbox/Patagonia/GIS South/Glaciers/dhdt_error/*.tif")

glacier_list = []

# Read rasters file
for glacier in list_files:
    glacier_i = riox.open_rasterio(glacier)
    glacier_i = glacier_i.rio.reproject("EPSG:32718", resampling = Resampling.bilinear)
    glacier_list.append(glacier_i)

# Merge/Mosaic multiple rasters using merge_arrays method of rioxarray
merged_raster = merge_arrays(dataarrays = glacier_list, res = (100, 100), crs="EPSG:32718", method='max')

# Save Raster to disk
merged_raster.rio.to_raster("/home/rooda/Dropbox/Patagonia/GIS South/Glaciers/dhdt_error_2021.tif")